In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import PIL                         
import io
import cv2  
import matplotlib.pyplot as plt
import plotly.express as px
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE

In [2]:
data_files = {"train": ["./Dataset/train/data-00000-of-00004.arrow","./Dataset/train/data-00001-of-00004.arrow","./Dataset/train/data-00002-of-00004.arrow","./Dataset/train/data-00003-of-00004.arrow"]}
my_dataset = load_dataset("arrow", data_files=data_files, split="train")
df=my_dataset.to_pandas()
df.head()

,image,age,gender,race,service_test
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,6,0,0,True
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,4,1,1,False
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1,1,2,False
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,1,1,True
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,1,1,True


In [3]:
x = []
y = []

for item in df["image"]:
    image_bytes = item['bytes']
    # Decodificar la imagen desde bytes
    image_array = np.frombuffer(image_bytes, dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    # Redimensionar la imagen a 48x48 píxeles
    image = cv2.resize(image, (96, 96))
    x.append(image)


y_age = df['age']
y_gender = df['gender']
y_race=df["race"]

x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(x, y_age, test_size=0.2, stratify=y_age)
x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(x, y_gender, test_size=0.2, stratify=y_gender)

In [4]:
agemodel = Sequential()
agemodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(96, 96, 3)))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Conv2D(64, (3,3), activation='relu'))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Conv2D(128, (3,3), activation='relu'))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Flatten())
agemodel.add(Dense(64, activation='relu'))
agemodel.add(Dropout(0.5))
agemodel.add(Dense(1, activation='relu'))

agemodel.compile(loss='mean_squared_error',
                 optimizer=optimizers.Adam(learning_rate=0.0001))

genmodel = Sequential()
genmodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(96, 96, 3)))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Conv2D(64, (3,3), activation='relu'))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Conv2D(128, (3,3), activation='relu'))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Flatten())
genmodel.add(Dense(64, activation='relu'))
genmodel.add(Dropout(0.5))
genmodel.add(Dense(1, activation='sigmoid'))

genmodel.compile(loss='binary_crossentropy',
                 optimizer=optimizers.Adam(learning_rate=0.0001),
                 metrics=['accuracy'])

racemodel=Sequential()
racemodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)))
racemodel.add(MaxPooling2D((2,2)))
racemodel.add(Conv2D(64, (3,3), activation='relu'))
racemodel.add(MaxPooling2D((2,2)))
racemodel.add(Conv2D(128, (3,3), activation='relu'))
racemodel.add(MaxPooling2D((2,2)))
racemodel.add(Flatten())
racemodel.add(Dense(64, activation='relu'))
racemodel.add(Dropout(0.5))
racemodel.add(Dense(1, activation='softmax'))

racemodel.compile(loss='categorical_crossentropy',
                 optimizer=optimizers.Adam(learning_rate=0.0001),
                 metrics=['accuracy'])

c:\Users\ma907\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
total_samples = len(y_train_age)
class_counts = y_train_age.value_counts()
class_weights = total_samples / (class_counts * len(class_counts))

In [6]:
x_train_age = np.array(x_train_age)
x_test_age = np.array(x_test_age)
        
batch_size = 16
   
train_datagen = ImageDataGenerator(rescale=1./255., width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
   
train_generator = train_datagen.flow(x_train_age, y_train_age, batch_size=batch_size)
test_generator = test_datagen.flow(x_test_age, y_test_age, batch_size=batch_size)
   
history1 = agemodel.fit(train_generator, epochs=32, shuffle=True, validation_data=test_generator,class_weight=class_weights) 
   

Epoch 1/32


c:\Users\ma907\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4338/4338 ━━━━━━━━━━━━━━━━━━━━ 195s 44ms/step - loss: 7.8939 - val_loss: 2.6340
Epoch 2/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 171s 39ms/step - loss: 6.4333 - val_loss: 2.8523
Epoch 3/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 170s 39ms/step - loss: 5.8221 - val_loss: 2.2107
Epoch 4/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 171s 39ms/step - loss: 5.1474 - val_loss: 2.0778
Epoch 5/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 186s 43ms/step - loss: 4.6255 - val_loss: 3.3737
Epoch 6/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 180s 41ms/step - loss: 4.2645 - val_loss: 2.5930
Epoch 7/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 175s 40ms/step - loss: 4.1902 - val_loss: 1.8729
Epoch 8/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 172s 40ms/step - loss: 3.9517 - val_loss: 2.3389
Epoch 9/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 169s 39ms/step - loss: 3.5832 - val_loss: 1.9875
Epoch 10/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 6457s 1s/step - loss: 3.4060 - val_loss: 1.8189
Epoch 11/32
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 296s 68ms/step - loss: 3.3831 - val_loss: 2.0844
Epoch 12/

In [8]:
agemodel.save("age_model_new.h5")

In [ ]:
x_train_gender=np.array(x_train_gender)
x_test_gender=np.array(x_test_gender)

batch_size=16
train_datagen = ImageDataGenerator(rescale=1./255., width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
   
train_generator = train_datagen.flow(x_train_gender, y_train_gender, batch_size=batch_size)
test_generator = test_datagen.flow(x_test_gender, y_test_gender, batch_size=batch_size)
   
history2 = genmodel.fit(train_generator, epochs=25, shuffle=True, validation_data=test_generator)

Epoch 1/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 205s 47ms/step - accuracy: 0.5971 - loss: 0.6588 - val_accuracy: 0.7234 - val_loss: 0.5448
Epoch 2/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 186s 43ms/step - accuracy: 0.7129 - loss: 0.5624 - val_accuracy: 0.7603 - val_loss: 0.4908
Epoch 3/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 178s 41ms/step - accuracy: 0.7429 - loss: 0.5178 - val_accuracy: 0.7828 - val_loss: 0.4549
Epoch 4/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 177s 41ms/step - accuracy: 0.7597 - loss: 0.4929 - val_accuracy: 0.7967 - val_loss: 0.4308
Epoch 5/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 176s 41ms/step - accuracy: 0.7796 - loss: 0.4645 - val_accuracy: 0.8111 - val_loss: 0.4066
Epoch 6/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 175s 40ms/step - accuracy: 0.7961 - loss: 0.4390 - val_accuracy: 0.8276 - val_loss: 0.3806
Epoch 7/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 713s 164ms/step - accuracy: 0.8030 - loss: 0.4224 - val_accuracy: 0.8390 - val_loss: 0.3624
Epoch 8/25
4338/4338 ━━━━━━━━━━━━━━━━━━━━ 3895s 898ms/step - accurac

In [13]:
def process_and_predict(image_array):
    
    img_pil = Image.fromarray(cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
    width, height = img_pil.size
    if width == height:
        img_pil = img_pil.resize((96,96), Image.Resampling.LANCZOS)
    else:
        if width > height:
            left = width//2 - height//2
            right = width//2 + height//2
            top = 0
            bottom = height
            img_pil = img_pil.crop((left,top,right,bottom))
            img_pil = img_pil.resize((96,96), Image.Resampling.LANCZOS)
        else:
            left = 0
            right = width
            top = 0
            bottom = width
            img_pil = img_pil.crop((left,top,right,bottom))
            img_pil = img_pil.resize((96,96), Image.Resampling.LANCZOS)
            
    ar = np.array(img_pil)
    ar = ar.astype('float32')
    ar /= 255.0
    ar = ar.reshape(-1, 96, 96, 3)
    
    age = agemodel.predict(ar)
    age = np.round(age)
    ar = ar.reshape(-1, 96, 96, 3)
    gender = np.round(genmodel.predict(ar))
    
    return int(age), int(gender)

In [9]:
fig = px.line(
    history1.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [31]:
fig = px.line(
    history2.history, y=['loss', 'val_loss','accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [ ]:
agemodel.save("agemodel.h5")
genmodel.save("genmodel.h5")

In [ ]:
import pickle   
with open("history1.pickle", "wb") as file:
    pickle.dump(history1.history, file)
    
with open("history2.pickle", "wb") as file:
    pickle.dump(history2.history, file)

In [11]:
data_files = {"validation": ["./Dataset/validation/data-00000-of-00001.arrow"]}
my_dataset = load_dataset("arrow", data_files=data_files, split="validation")
df=my_dataset.to_pandas()
print(df)

                                                   image  age  gender  race  \
0      {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    1       0     0   
1      {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    6       1     0   
2      {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    4       0     3   
3      {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    3       1     5   
4      {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    3       0     6   
...                                                  ...  ...     ...   ...   
10949  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    4       0     3   
10950  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    6       0     3   
10951  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    7       0     5   
10952  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    3       1     0   
10953  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    5       0     5   

       service_test  
0             False  
1      

In [14]:
y_true_age = []
y_pred_age = []
y_true_gender = []
y_pred_gender = []

# Iterar sobre las imágenes del dataset
for i in range(len(df)):
    img_bytes = df['image'].iloc[i]['bytes']
    img_array = np.frombuffer(img_bytes, np.uint8)
    img_array = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # Obtener los valores reales de la imagen
    true_age = df['age'].iloc[i]  # Reemplaza 'age' con la columna correcta en tu DataFrame
    true_gender = df['gender'].iloc[i] # Reemplaza 'gender' con la columna correcta en tu DataFrame

    # Procesar la imagen y obtener las predicciones
    pred_age, pred_gender = process_and_predict(img_array)

    # Almacenar los valores reales y las predicciones
    y_true_age.append(true_age)
    y_pred_age.append(pred_age)
    y_true_gender.append(true_gender)
    y_pred_gender.append(pred_gender)
    
print(pred_age)
print(true_age)

# Calcular las métricas
accuracy_age = accuracy_score(y_true_age, y_pred_age)
f1_age = f1_score(y_true_age, y_pred_age, average='weighted')
recall_age = recall_score(y_true_age, y_pred_age, average='weighted')
confusion_matrix_age = confusion_matrix(y_true_age, y_pred_age)

accuracy_gender = accuracy_score(y_true_gender, y_pred_gender)
f1_gender = f1_score(y_true_gender, y_pred_gender, average='weighted')
recall_gender = recall_score(y_true_gender, y_pred_gender, average='weighted')
confusion_matrix_gender = confusion_matrix(y_true_gender, y_pred_gender)

# Imprimir los resultados
print("===== Resultados de la Edad =====")
print("Precisión:", accuracy_age)
print("F1-score:", f1_age)
print("Recall:", recall_age)
print("Matriz de confusión:")
print(confusion_matrix_age)

print("n===== Resultados del Género =====")
print("Precisión:", accuracy_gender)
print("F1-score:", f1_gender)
print("Recall:", recall_gender)
print("Matriz de confusión:")
print(confusion_matrix_gender)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

c:\Users\ma907\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [18]:
age_report=classification_report(y_true_age, y_pred_age)
print(age_report)

              precision    recall  f1-score   support

           0       0.55      0.18      0.27       199
           1       0.66      0.11      0.18      1356
           2       0.17      0.07      0.10      1181
           3       0.34      0.14      0.20      3300
           4       0.23      0.40      0.29      2330
           5       0.18      0.54      0.27      1353
           6       0.21      0.17      0.19       796
           7       0.26      0.03      0.06       321
           8       0.86      0.05      0.10       118
           9       0.00      0.00      0.00         0

    accuracy                           0.23     10954
   macro avg       0.35      0.17      0.16     10954
weighted avg       0.31      0.23      0.21     10954



c:\Users\ma907\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

c:\Users\ma907\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

c:\Users\ma907\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

